<a href="https://colab.research.google.com/github/benjaminnigjeh/keyProteoforms/blob/main/VAEdatasetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation of external packages

In [ ]:
!apt-get update -qq
!apt-get install -y -qq mono-complete
!pip install -qq fisher-py


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


#Import packages

In [ ]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import subprocess
import os

#Helper functions

In [ ]:
def quantitativeDeconvolution(folder_path, file_path):
    os.chdir(folder_path)
    def helper_regex(text):
      match = re.search(rf"{'Full'}\s+(\w+)", text)
      if match:
        return match.group(1)
      return None
    raw = RawFile(file_path)
    dir_name = file_path.split('/')[-1].split('.')[0]
    new_dir = os.path.join(os.getcwd(), dir_name)
    if not os.path.exists(new_dir): os.mkdir(new_dir)
    os.chdir(new_dir)
    scans = []
    for i in range(1, raw.number_of_scans):
            raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
            if str(helper_regex(raw_scan.scan_type)) == 'ms':
                    scans.append(i)
    for i in tqdm(scans):
        raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
        scan_masses = raw_scan.preferred_masses
        scan_intensities = raw_scan.preferred_intensities
        data = {
                'Column1': scan_masses,
                'Column2': scan_intensities}
        df = pd.DataFrame(data)
        file_name = str(i) + '.txt'
        df.to_csv(file_name, sep='\t', index=False)
        command = f"python -m unidec -f {file_name}"
        subprocess.run(command, shell=True)

    x_value = list(range(5000, 21000, 10))
    y_value = [0]*len(x_value)
    for i in scans:
        print(i)
        os.chdir(new_dir)
        os.chdir(str(i) + '_unidecfiles')
        try:
            data = np.loadtxt(str(i) + '_mass.txt')
            x = data[:, 0]  # First column (x-values)
            y = data[:, 1]  # Second column (y-values)
            for j in range(0, len(x)):
                 if x[j] in x_value:
                      index = x_value.index(x[j])
                      if index in range(len(y_value)):
                           y_value[index] = y_value[index] + y[j]

        except FileNotFoundError: print("File not found. Skipping...")
    data = {
                'Column1': x_value,
                'Column2': y_value}
    df = pd.DataFrame(data)
    os.chdir(folder_path)
    file_name = file_path[:-4] + '.txt'
    df.to_csv(file_name, sep='\t', index=False)

    return()

def MS1quan(folder_path, file_path):
    os.chdir(folder_path)
    def helper_regex(text):
      match = re.search(rf"{'Full'}\s+(\w+)", text)
      if match:
        return match.group(1)
      return None
    raw = RawFile(file_path)
    data_intensities = [0]*1369
    for i in tqdm(range(1, raw.number_of_scans)):
                raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
                if str(helper_regex(raw_scan.scan_type)) == 'ms':
                        scan_masses = raw_scan.preferred_masses
                        scan_intensities = raw_scan.preferred_intensities
                        for j in range(0,len(scan_masses)):
                                index = round(scan_masses[j])
                                if index > 600 and index < 1969:
                                        data_intensities[index-600] = scan_intensities[j] + data_intensities[index-600]
    return(data_intensities)

#Quantitative deconvolution: scan-by-scan deconvolution

In [ ]:
folder_path = 'D:/conv/samples'
os.chdir(folder_path)

file_path_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

for file_path in file_path_list:
    quantitativeDeconvolution(folder_path, file_path)


#Quantitative deconvolution: data aggregation

In [ ]:
folder_path = 'D:/conv/results'
os.chdir(folder_path)

file_path_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
combined_result = []
for file_txt in file_path_list:
    data = np.genfromtxt(str(file_txt))
    combined_result.append(data[:, 1])

#Quantitative deconvolution: data export

In [ ]:
df = pd.DataFrame(combined_result)

column_max = df.max()
df_normalized = df / (column_max + 0.001)

df_normalized.to_csv('D:/combined_result.csv')

#MS1 quantification: data aggregation

In [ ]:
folder_path = 'D:/conv/samples'
os.chdir(folder_path)

file_path_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

result = []

for file_path in file_path_list[57]:
    result.append(MS1quan(folder_path, file_path))




#MS1 quantification: data export

In [ ]:
df = pd.DataFrame(result)

column_max = df.max()
df_normalized = df / (column_max + 0.001)

df_normalized.to_csv('D:/ms1_result.csv')